In [1]:
def generate_class_labels(labels):

    unique_labels = list(set(labels))

    unique_labels_map = {}

    for index, label in enumerate(unique_labels, start=0):
        unique_labels_map[label] = index

    return unique_labels_map

In [2]:
VAR = {
    'max_len': 128,
    'batch_size': 50
}

In [3]:
import numpy
import pandas
import torch
import torch.nn as nn
from transformers import BertTokenizer, AdamW, BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

C:\Users\JS\AppData\Local\Temp\ipykernel_29080\4211774795.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


# Classification Task: Initiating Data

In [4]:
resume_df = pandas.read_csv("cleanedResumes.csv")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# labels = resume_df['Category']
# label_map = generate_class_labels(labels)
# 
# resume_df['Category'] = resume_df['Category'].apply(lambda label_name: label_map[label_name])

In [5]:
type(resume_df['Category'].iloc[0])

str

In [6]:
encoder = LabelEncoder()

In [7]:
resume_df['Category'] = encoder.fit_transform(resume_df['Category'])
resume_df['Category'] = resume_df['Category'].astype(numpy.int64)

In [8]:
type(resume_df['Category'].iloc[0])

numpy.int64

In [9]:
resume_df['Category'].value_counts().max()

14

In [10]:
import random

# minimum = resume_df['Category'].value_counts().max()
minimum = 0
current_lowest = resume_df['Category'].value_counts().min()

#Check
count = resume_df['Category'].value_counts()
remaining = 7 - count[count<minimum]

while len(remaining != 0):
    count = resume_df['Category'].value_counts()
    remaining = 7 - count[count<minimum]

    for category in remaining.index:
        someInt = random.randint(0, current_lowest-1)
        value_to_append = resume_df[
            resume_df['Category']==category
            ]['Resume'].values[someInt]


        df_to_concat = pandas.DataFrame({
            'Category': [category],
            'Resume': [value_to_append]
        })

        resume_df = pandas.concat([resume_df, df_to_concat], axis=0)

In [11]:
resume_df['Category'].value_counts()

Category
15    14
6     12
12    12
7     11
0     10
9      8
13     8
8      8
4      8
23     8
5      7
21     7
14     7
20     7
1      7
2      7
11     6
22     6
17     6
10     6
16     5
24     5
3      5
18     4
19     4
Name: count, dtype: int64

In [12]:
resumes = resume_df.Resume.values
resumes = ["[CLS] " + resume + " [SEP]" for resume in resumes]
labels = resume_df.Category.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(resume) for resume in resumes]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=VAR['max_len'], dtype="long", truncating="post", padding="post")

attention_masks = []
for sequence in input_ids:
    sequence_mask = [float(id > 0) for id in sequence]
    attention_masks.append(sequence_mask)

In [13]:
training_inputs, validation_test_inputs, training_labels, validation_test_labels, training_masks, validation_test_masks = train_test_split(
    input_ids, labels, attention_masks,
    random_state=42, test_size=0.3
)

validation_inputs, testing_inputs, validation_labels, testing_labels, validation_masks, testing_masks = train_test_split(
    validation_test_inputs, validation_test_labels, validation_test_masks, random_state=42, test_size=0.3
)

training_data = TensorDataset(torch.tensor(training_inputs), torch.tensor(training_masks), torch.tensor(training_labels))
training_sampler = RandomSampler(training_data)
training_dataloader = DataLoader(training_data, sampler=training_sampler, batch_size=VAR['batch_size'])

validation_data = TensorDataset(torch.tensor(validation_inputs), torch.tensor(validation_masks), torch.tensor(validation_labels))
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=VAR['batch_size'])


In [14]:
# len(encoder.classes_)

In [15]:
from transformers import BertConfig, BertModel
configuration = BertConfig()
model = BertModel(configuration)
configuration = model.config

from transformers import BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=len(label_map.keys()))
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(encoder.classes_))
model = nn.DataParallel(model)
model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\JS\anaconda3\envs\NLPProjectVenv\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
def flat_accuracy(predicted_labels, labels):
    predicted_labels = numpy.argmax(predicted_labels.to('cpu').numpy(), axis=1).flatten()
    labels = labels.to('cpu').numpy().flatten()
    return numpy.sum(predicted_labels == labels) / len(labels)


In [20]:
from tqdm import trange
epochs = 200
random.seed(965)
training_losses = []

for epoch in trange(epochs, desc="Epoch"):
    model.train()
    training_loss = 0
    training_steps = 0

    for step, batch in enumerate(training_dataloader):
        inputs = batch[0].to(device)
        attention_masks = batch[1].to(device)
        # labels = labels.type(torch.LongTensor)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_masks, labels=labels)
        # print("TRAIN", outputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        training_loss += loss.item()
        training_steps += 1

        training_losses.append(loss.item())

    average_training_loss = training_loss/training_steps
    print("Epoch {}: Average Training Loss: {}".format(epoch+1, average_training_loss))

    model.eval()
    validation_accuracy = 0
    validation_steps = 0

    for batch in validation_dataloader:
        inputs = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(inputs, attention_mask=attention_masks, labels=labels)
            # print("VAL", outputs)

        logits = outputs.logits
        temp_validation_accuracy = flat_accuracy(logits, labels)
        validation_accuracy += temp_validation_accuracy
        validation_steps += 1

    average_validation_accuracy = validation_accuracy/validation_steps
    print("Epoch {}: Validation Accuracy: {}".format(epoch+1, average_validation_accuracy))

Epoch:   0%|          | 1/200 [00:01<03:52,  1.17s/it]

Epoch 1: Average Training Loss: 0.021061406781276066
Epoch 1: Validation Accuracy: 0.38461538461538464


Epoch:   1%|          | 2/200 [00:02<03:16,  1.01it/s]

Epoch 2: Average Training Loss: 0.020489394664764404
Epoch 2: Validation Accuracy: 0.38461538461538464


Epoch:   2%|▏         | 3/200 [00:02<03:08,  1.05it/s]

...

Epoch 198: Average Training Loss: 0.007860712241381407
Epoch 198: Validation Accuracy: 0.4358974358974359


Epoch: 100%|█████████▉| 199/200 [02:53<00:00,  1.14it/s]

Epoch 199: Average Training Loss: 0.007822852271298567
Epoch 199: Validation Accuracy: 0.4358974358974359


Epoch: 100%|██████████| 200/200 [02:54<00:00,  1.15it/s]

Epoch 200: Average Training Loss: 0.007961446729799112
Epoch 200: Validation Accuracy: 0.4358974358974359


In [21]:
input_ids = torch.tensor(testing_inputs)
attention_masks = torch.tensor(testing_masks)
labels = torch.tensor(testing_labels)

prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_dataloader = DataLoader(prediction_data, batch_size=VAR['batch_size'])

model.eval()
logits_set = []
labels_set = []

for batch in prediction_dataloader:
    batch_input_ids, batch_attention_masks, batch_labels = batch
    batch_input_ids, batch_attention_masks, batch_labels = batch_input_ids.to(device), batch_attention_masks.to(device), batch_labels.to(device)

    with torch.no_grad():
        outputs = model(batch_input_ids, attention_mask=batch_attention_masks)
        logits = outputs.logits

    logits_set.append(logits.cpu().numpy())
    labels_set.append(batch_labels.cpu().numpy())

In [28]:
from sklearn.metrics import matthews_corrcoef
matthews_set = []

# Calculate Matthews correlation coefficient for each batch
for i in range(len(labels_set)):
    mcc = matthews_corrcoef(labels_set[i], numpy.argmax(logits_set[i], axis=1).flatten())
    matthews_set.append(mcc)

for i, mcc in enumerate(matthews_set):
    print(f"Batch {i + 1}: MCC = {mcc}")

# Calculate the overall Matthews correlation coefficient
overall_mcc = numpy.mean(matthews_set)
print(f"\nOverall MCC: {overall_mcc}")

Batch 1: MCC = 0.2917299829957891

Overall MCC: 0.2917299829957891


In [18]:
from sklearn.metrics import classification_report

